# Regression LASSO (coordinate descent)

In [3]:
import graphlab
import numpy as np

# Load in house sales data

In [4]:
# Housing sales data for King County, Seatle, WA
sales = graphlab.SFrame('kc_house_data.gl/')
sales['floors'] = sales['floors'].astype(int) 

[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: C:\Users\Darshan\AppData\Local\Temp\graphlab_server_1486243092.log.0


This non-commercial license of GraphLab Create for academic use is assigned to darshanb@umd.edu and will expire on January 08, 2018.


In [5]:
def get_numpy_data(df,features,output):
    df['constant']=1 # A new column for the constant/intercept term
    features=['constant']+features # Column Names
    sframe=df[features] # Copy all the data for the selected features in the new dataframe
    smatrix=sframe.to_numpy() # Creating a numpy array for the features
    output_array=np.array(df[output])# Array for the ouptut/target
    return smatrix, output_array

In [6]:
def predict_output(f, w):
    # This fucntion will multiply the features by its weights
    # So if there are D features, a weight is assigned to each feature
    pred_out=np.dot(f,w)
    return pred_out

# Normalize features
In the house dataset, features vary wildly in their relative magnitude: `sqft_living` is very large overall compared to `bedrooms`, for instance. As a result, weight for `sqft_living` would be much smaller than weight for `bedrooms`. This is problematic because "small" weights are dropped first as `l1_penalty` goes up. 

To give equal considerations for all features, we need to **normalize features**: we divide each feature by its 2-norm so that the transformed feature has norm 1.

Let's see how we can do this normalization easily with Numpy: let us first consider a small matrix.

In [7]:
def normalize_features(features):
    #To normalize the dataset
    norms = np.linalg.norm(features, axis=0)
    normalized_features=features/norms
    return (normalized_features, norms)

# Implementing Coordinate Descent with normalized features

We seek to obtain a sparse set of weights by minimizing the LASSO cost function
```
SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|).
```
(By convention, we do not include `w[0]` in the L1 penalty term. We never want to push the intercept to zero.)

The absolute value sign makes the cost function non-differentiable, so simple gradient descent is not viable (you would need to implement a method called subgradient descent). Instead, we will use **coordinate descent**: at each iteration, we will fix all weights but weight `i` and find the value of weight `i` that minimizes the objective. That is, we look for
```
argmin_{w[i]} [ SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|) ]
```
where all weights other than `w[i]` are held to be constant. We will optimize one `w[i]` at a time, circling through the weights multiple times.  
  1. Pick a coordinate `i`
  2. Compute `w[i]` that minimizes the cost function `SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|)`
  3. Repeat Steps 1 and 2 for all coordinates, multiple times

For this notebook, we use **cyclical coordinate descent with normalized features**, where we cycle through coordinates 0 to (d-1) in order, and assume the features were normalized as discussed above. The formula for optimizing each coordinate is as follows:
```
       ┌ (ro[i] + lambda/2)     if ro[i] < -lambda/2
w[i] = ├ 0                      if -lambda/2 <= ro[i] <= lambda/2
       └ (ro[i] - lambda/2)     if ro[i] > lambda/2
```
where
```
ro[i] = SUM[ [feature_i]*(output - prediction + w[i]*[feature_i]) ].
```

Note that we do not regularize the weight of the constant feature (intercept) `w[0]`, so, for this weight, the update is simply:
```
w[0] = ro[i]
```

## Effect of L1 penalty

Let us consider a simple model with 2 features:

In [10]:
#Initial parameters
simple_features = ['sqft_living', 'bedrooms']
my_output = 'price'
weights = np.array([1., 4., 1.])
(simple_feature_matrix, output) = get_numpy_data(sales, simple_features, my_output) # Get the features in array


#Normailze the features
simple_feature_matrix, norms = normalize_features(simple_feature_matrix) 

#Predictions
prediction = predict_output(simple_feature_matrix,weights)

Compute the values of `ro[i]` for each feature in this simple model, using the formula given above, using the formula:
```
ro[i] = SUM[ [feature_i]*(output - prediction + w[i]*[feature_i]) ]
```

In [11]:
ro=[0,0,0]
for i in range(len(weights)):
    ro[i]=np.sum(np.dot(simple_feature_matrix[:,i],(output-prediction+weights[i]*simple_feature_matrix[:,i])))

Whenever `ro[i]` falls between `-l1_penalty/2` and `l1_penalty/2`, the corresponding weight `w[i]` is sent to zero. Now suppose we were to take one step of coordinate descent on either feature 1 or feature 2. What range of values of `l1_penalty` **would not** set `w[1]` zero, but **would** set `w[2]` to zero, if we were to take a step in that coordinate? 

## Single Coordinate Descent Step

Implement coordinate descent that minimizes the cost function over a single feature i. Note that the intercept (weight 0) is not regularized. The function should accept feature matrix, output, current weights, l1 penalty, and index of feature to optimize over. The function should return new weight for feature i.

In [12]:
def lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty):
    # compute prediction
    prediction =np.dot(feature_matrix,weights)
    # compute ro[i] = SUM[ [feature_i]*(output - prediction + weight[i]*[feature_i]) ]
    ro_i = np.sum(np.dot(feature_matrix[:,i],(output-prediction+weights[i]*feature_matrix[:,1])))
    if i == 0: # intercept -- do not regularize
        new_weight_i = ro_i 
    elif ro_i < -l1_penalty/2.:
        new_weight_i = ro_i+l1_penalty/2.0
    elif ro_i > l1_penalty/2.:
        new_weight_i =ro_i-l1_penalty/2.0
    else:
        new_weight_i = 0.
    
    return new_weight_i

## Cyclical coordinate descent 

Now that we have a function that optimizes the cost function over a single coordinate, let us implement cyclical coordinate descent where we optimize coordinates 0, 1, ..., (d-1) in order and repeat.

When do we know to stop? Each time we scan all the coordinates (features) once, we measure the change in weight for each coordinate. If no coordinate changes by more than a specified threshold, we stop.

For each iteration:
1. As you loop over features in order and perform coordinate descent, measure how much each coordinate changes.
2. After the loop, if the maximum change across all coordinates is falls below the tolerance, stop. Otherwise, go back to step 1.

Return weights

**IMPORTANT: when computing a new weight for coordinate i, make sure to incorporate the new weights for coordinates 0, 1, ..., i-1. One good way is to update your weights variable in-place. See following pseudocode for illustration.**
```
for i in range(len(weights)):
    old_weights_i = weights[i] # remember old value of weight[i], as it will be overwritten
    # the following line uses new values for weight[0], weight[1], ..., weight[i-1]
    #     and old values for weight[i], ..., weight[d-1]
    weights[i] = lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty)
    
    # use old_weights_i to compute change in coordinate
    ...
```

In [13]:
def lasso_cyclical_coordinate_descent(feature_matrix, output, initial_weights, l1_penalty, tolerance):
    weights=initial_weights.copy()
    flag=False
    while not flag:
        for i in range(len(weights)):
            max_change=0
            old_weights=weights[i]
            weights[i]=lasso_coordinate_descent_step(i,feature_matrix,output,weights,l1_penalty)
            if (np.abs(old_weights-weights[i])<tolerance):
                flag=True
            else:
                max_change=np.abs(old_weights-weights[i])
    return weights

Using the following parameters, learn the weights on the sales dataset. 

In [14]:
# Sample regression with sqft_living and bedrooms as features and price as targets
simple_features = ['sqft_living', 'bedrooms']
my_output = 'price'

#Intial parameters
initial_weights = np.zeros(3)
l1_penalty = 1e7
tolerance = 1.0

(simple_feature_matrix, output) = get_numpy_data(sales, simple_features, my_output)# Get the features in array

(normalized_simple_feature_matrix, simple_norms) = normalize_features(simple_feature_matrix) # normalize features

In [19]:
weights = lasso_cyclical_coordinate_descent(normalized_simple_feature_matrix, output,
                                            initial_weights, l1_penalty, tolerance)
print "The constant term is ", weights[0]
print "The coefficient of sqft_living is:",weights[1]
print "The co-efficient of bedrooms is:", weights[2]

The constant term is  50994436.9598
The coefficient of sqft_living is: 36290522.5719
The co-efficient of bedrooms is: 0.0


In [23]:
predictions = predict_output(normalized_simple_feature_matrix, weights) # Predict the output
rss=np.sum(np.square(predictions-output)) # Calculate the RSS
print "The RSS of the model is:",(rss)

The RSS of the model is: 2.03990281919e+15


# Evaluating LASSO fit with more features

Let us split the sales dataset into training and test sets.

In [24]:
train_data,test_data = sales.random_split(.8,seed=0)

Let us consider the following set of features.

In [25]:
all_features = ['bedrooms',
                'bathrooms',
                'sqft_living',
                'sqft_lot',
                'floors',
                'waterfront', 
                'view', 
                'condition', 
                'grade',
                'sqft_above',
                'sqft_basement',
                'yr_built', 
                'yr_renovated']

In [26]:
(multiple_feature_matrix, output) = get_numpy_data(train_data, all_features,  my_output) # Get the features in array

(normalized_multiple_feature_matrix,multiple_norms) = normalize_features(multiple_feature_matrix) # normalize features

In [37]:
#Inital parameters
init=np.array(np.zeros(14))
weights1e7 = lasso_cyclical_coordinate_descent(normalized_multiple_feature_matrix,output,
                                            init, 1e7, 1)
for i in range(len(all_features)+1):
    if i==0:
        print "The co-efficient for constant term is", weights1e7[i]
    else:
        print "The co-efficient for",all_features[i-1],"is" ,weights1e7[i]

The co-efficient for constant term is 71114625.7528
The co-efficient for bedrooms is 0.0
The co-efficient for bathrooms is 3743972.43192
The co-efficient for sqft_living is 5271064.34696
The co-efficient for sqft_lot is 0.0
The co-efficient for floors is 0.0
The co-efficient for waterfront is 7173100.28481
The co-efficient for view is 7025132.06643
The co-efficient for condition is -5530804.65692
The co-efficient for grade is 0.0
The co-efficient for sqft_above is 394565.584395
The co-efficient for sqft_basement is 2242690.39485
The co-efficient for yr_built is -2160960.47386
The co-efficient for yr_renovated is 0.0


Next, learn the weights with `l1_penalty=1e8`, on the training data. Initialize weights to all zeros, and set the `tolerance=1`.  Call resulting weights `weights1e8`, you will need them later.

In [39]:
#Inital parameters
init=np.array(np.zeros(14))
weights1e8 = lasso_cyclical_coordinate_descent(normalized_multiple_feature_matrix,output,
                                            init, 1e8, 1)
for i in range(len(all_features)+1):
    if i==0:
        print "The co-efficient for constant term is", weights1e8[i]
    else:
        print "The co-efficient for",all_features[i-1],"is" ,weights1e8[i]

The co-efficient for constant term is 71114625.7528
The co-efficient for bedrooms is 0.0
The co-efficient for bathrooms is 0.0
The co-efficient for sqft_living is 0.0
The co-efficient for sqft_lot is 0.0
The co-efficient for floors is 0.0
The co-efficient for waterfront is 0.0
The co-efficient for view is 0.0
The co-efficient for condition is 0.0
The co-efficient for grade is 0.0
The co-efficient for sqft_above is 0.0
The co-efficient for sqft_basement is 0.0
The co-efficient for yr_built is 0.0
The co-efficient for yr_renovated is 0.0


In [43]:
#Intialize the parameters
init=np.array(np.zeros(14))
weights1e4 = lasso_cyclical_coordinate_descent(normalized_multiple_feature_matrix,output,
                                            init, 1e4, 1)

for i in range(len(all_features)+1):
    if i==0:
        print "The co-efficient for constant term is", weights1e4[i]
    else:
        print "The co-efficient for",all_features[i-1],"is" ,weights1e4[i]


The co-efficient for constant term is -4219567.74696
The co-efficient for bedrooms is -11355341.9608
The co-efficient for bathrooms is 11709571.1771
The co-efficient for sqft_living is 33240203.2092
The co-efficient for sqft_lot is -474676.536084
The co-efficient for floors is -1779154.46793
The co-efficient for waterfront is 4085908.96158
The co-efficient for view is 5426533.38329
The co-efficient for condition is 3999062.16521
The co-efficient for grade is 26920741.2065
The co-efficient for sqft_above is 20802789.2176
The co-efficient for sqft_basement is 5478519.62255
The co-efficient for yr_built is -7895142.85166
The co-efficient for yr_renovated is 2035348.78783


## Thank You!